# Fact Extraction Model

In [24]:
OLLAMA_MODEL = "llama3.2"
# OLLAMA_MODEL = "llama2"

OLLAMA_HOST = "http://host.docker.internal:11434" # for when running within docker image
# OLLAMA_HOST = "http://localhost:11434"

In [71]:
!pip install -r requirements.txt -q

import requests
import asyncio
import aiohttp
import nest_asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import nest_asyncio
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
from langchain.prompts import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease  
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 1s (449 kB/s)   
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.18).
unzip is already the newest version (6.0-26ubuntu3.2).
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 74 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libxcomposite-dev is already the newest version (1:0.4.5-1build2).
libxdamage-dev is already the newest version (1:1.1.5-2build2).
libxrandr-dev is already the newest version (2:1.5.2-1build1).
libgbm-dev is already the newest versi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
nest_asyncio.apply()

ollama_model = Ollama(model=OLLAMA_MODEL, base_url=OLLAMA_HOST)

In [64]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate

# claim_parse_system_prompt = "You are a tool to extract notable claims from text and return those excerpts alongside the claim being made, with clear subjects and predicates. In the event of no claims, just return '[NA]'."
claim_parse_example_prompt = PromptTemplate.from_template("Text: {input}\nOutput:\n{response}")
claim_parse_examples = [{
    "input": """The New York Times Tech Guild ended its unfair labor practice strike Monday, one week after a majority of the 600-person unit walked off the job just before Election Day.""",
    "response": """'The New York Times Tech Guild ended its unfair labor practice strike Monday': The New York Times Tech Guild ended its strike Monday."""
},
{
    "input": "@realDonaldTrump: Lyin’ Kamala is giving a News Conference now, saying that I want to end the Affordable Care Act. I never mentioned doing that, never even thought about such a thing. She also said I want to end Social Security. Likewise, never mentioned it, or thought of it. She is the one that wants to end Social Security and, she will do it, by putting the millions of Migrants coming into our Country into it. Kamala is a LIAR! Everything that comes out of her mouth is a LIE. It’s MADE UP FICTION, and she’s doing it because she’s losing, and losing BIG!",
    "response" : """Excerpt: 'Lyin’ Kamala is giving a News Conference now, saying that I want to end the Affordable Care Act.'\nRestatement: Kamala Harris has said that Donald Trump wants to end the Affordable Care Act.
Excerpt: 'I never mentioned doing that, never even thought about such a thing'\nRestatement: Donald Trump never said he wanted to end the Affordable Care Act.
Excerpt: 'She also said I want to end Social Security.'\nRestatement: Kamala Harris said that Donald Trump wants to end Social Security.
Excerpt: 'Likewise, never mentioned it, or thought of it'\nRestatement: Donald Trump never mentioned he wanted to end Social Security.
Excerpt: 'She is the one that wants to end Social Security and, she will do it'\nRestatement: Kamala Harris will end Social Security.
Excerpt: 'by putting the millions of Migrants coming into our Country into it.'\nRestatement: Kamala Harris plans to add millions of migrants to Social Security.
Excerpt: 'she’s losing, and losing BIG!'\nRestatement: Kamala Harris is losing the election."""
},
{
    "input": """
        Blooming flower buds next to trees with changing leaves. Fans and air conditioners working overtime. Iced coffees instead of hot ones. Temperatures more like Florida than the Mid-Atlantic. The weather has been weirdly warm in the eastern two-thirds of the United States as November gets started — more than 10 degrees above average across parts of over 20 states.

        Want to know how your actions can help make a difference for our planet? Sign up for the Climate Coach newsletter, in your inbox every Tuesday and Thursday.
        Conditions in the Rockies and on the West Coast have balanced out some of the unusual eastern warmth, with cooler-than-average temperatures and a big early-season dump of snow.

        But in the eastern half of the nation, will you eventually be able to pull out your hats and gloves this coming winter? Or is the subtropical weather here to stay?
    """,
    "response": """Excerpt: 'The weather has been weirdly warm in the eastern two-thirds of the United States as November gets started — more than 10 degrees above average across parts of over 20 states.'\nRestatement: The weather has been 10 degrees above average across parts of 20 states.
Excerpt: 'Conditions in the Rockies and on the West Coast have balanced out some of the unusual eastern warmth, with cooler-than-average temperatures and a big early-season dump of snow.'\nRestatement: The Rockies have had cooler-than-average temperatures and large, early amounts of snow."""
},
{
    "input": """
        Silent snowfall drifts,
        blanketing the world in white,
        winter’s breath, so still.
    """,
    "response": """No notable claims found."""
},
{
    "input": """What are the most effective strategies for enhancing productivity in a remote work environment? How do different communication styles impact team dynamics? Is it possible to boost creativity through structured brainstorming sessions, or do they stifle innovation? What role does technology play in maintaining work-life balance? Should companies prioritize flexibility over traditional work hours, and what are the potential downsides of doing so? How can organizations ensure that they are fostering an inclusive culture for employees from diverse backgrounds? What metrics should be used to evaluate employee satisfaction effectively? Is there a way to quantify the long-term benefits of professional development programs? How can leaders balance transparency with confidentiality in decision-making?""",
    "response": """No notable claims found."""
},
{
    "input": """
        Eboni Thompson’s Instagram isn’t just an ode to the classic literature that she loves. It’s also a celebration of annotation: Warmly lit video reels linger on the colorful tabs that fringe the edges of her Penguin Classics, the ruler-straight lines highlighting important passages and the margins where neat handwriting frames each page.

        Annotations have traditionally been the domain of academics, used to add commentary, feedback and criticism to texts. Annotation is “elemental to scholarship,” according to Remi Kalir and Antero Garcia, who literally wrote the book on annotation (“Annotation,” MIT Press). As a reminder of the high-minded history of marginalia, they note that Milton annotated Shakespeare.
    """,
    "response": """Excerpt: 'Annotation is “elemental to scholarship,” according to Remi Kalir and Antero Garcia'\nRestatement: Remi Kalir and Antero Garcia said that annotation is elemental to scholarship.
Excerpt: 'who literally wrote the book on annotation (“Annotation,” MIT Press)'\nRestatement: Remi Kalir and Antero Garcia wrote a book on annotation."""
},
{
    "input": """BREAKING NEWS: BEYONCE WAS INVOLVED IN A HORRIFIC CAR CRASH! #RIPBEYONCE""",
    "response": """Excerpt:'BEYONCE WAS INVOLVED IN A HORRIFIC CAR CRASH! #RIPBEYONCE'\nRestatement: Beyonce has died in a car crash."""
}]

claim_parse_prompt = FewShotPromptTemplate(
    examples=claim_parse_examples,
    example_prompt=claim_parse_example_prompt,
    prefix="""You are an advanced language model tasked with identifying notable claims within a given text. For each claim, return:
1. The exact excerpt where the claim is made.
2. A summary of the claim, clarifying the main assertion.

Notable claims include:
- Allegations about events or actions.
- Statistics, figures, or specific measurements cited.
- Attributed quotes or opinions linked to a particular person or group.
- Predictions or assertions that could impact public perception or lead to real-world consequences if inaccurate.

If there are no notable claims in the text, return: "No notable claims found." Below are a series of examples.
""",
    suffix="\n\nNow, process the following text accordingly.\n\nText: {input}",
    input_variables=["input"],
)

In [66]:
print(
claim_parse_prompt.invoke({"input": """“Did you know the Kamala price hikes have cost the average American family $28,000?”
— Donald Trump, in a TikTok video, Aug. 15
Get the latest election news and results
“As a result of Kamala’s inflation price hikes, they’ve cost the typical household a total of $28,000. These are numbers coming from the government. They are not coming from me.”
— Trump, media event in Bedminster, N.J., Aug. 15"""}).to_string()
)

You are an advanced language model tasked with identifying notable claims within a given text. For each claim, return:
1. The exact excerpt where the claim is made.
2. A summary of the claim, clarifying the main assertion.

Notable claims include:
- Allegations about events or actions.
- Statistics, figures, or specific measurements cited.
- Attributed quotes or opinions linked to a particular person or group.
- Predictions or assertions that could impact public perception or lead to real-world consequences if inaccurate.

If there are no notable claims in the text, return: "No notable claims found." Below are a series of examples.


Text: The New York Times Tech Guild ended its unfair labor practice strike Monday, one week after a majority of the 600-person unit walked off the job just before Election Day.
Output:
'The New York Times Tech Guild ended its unfair labor practice strike Monday': The New York Times Tech Guild ended its strike Monday.

Text: @realDonaldTrump: Lyin’ Kamala i

In [67]:
%%time 
new_input = "Other economists were skeptical of Antoni’s math — a combination of two calculations involving purchasing and borrowing costs — and pointed to more reliable metrics, such as real disposable personal income per capita. Many such statistics are affected by the pandemic, adding to the complexity. But real disposable income per capita now is up about $3,000 since March 2020, just before temporary covid relief payments start showing up in the data and skew it for more than a year."

# Define the chain with LangChain’s LLMChain
few_shot_claim_chain = LLMChain(llm=ollama_model, prompt=claim_parse_prompt)

# Example of querying through the chain
response = few_shot_claim_chain.run(input=new_input)
print(response)

Here are the notable claims with their corresponding excerpts and restatements:

Excerpt: 'Other economists were skeptical of Antoni’s math — a combination of two calculations involving purchasing and borrowing costs'
Restatement: Some economists questioned the math used by Antoni.

Excerpt: 'and pointed to more reliable metrics, such as real disposable personal income per capita.'
Restatement: Economists suggested using more reliable metrics, including real disposable personal income per capita.

Excerpt: 'Many such statistics are affected by the pandemic'
Restatement: Many statistics are impacted by the pandemic.

Excerpt: 'But real disposable income per capita now is up about $3,000 since March 2020'
Restatement: Real disposable income per capita has increased by approximately $3,000 since March 2020.
CPU times: user 34.6 ms, sys: 1.82 ms, total: 36.4 ms
Wall time: 1.68 s


In [68]:
quoted_input = """“Did you know the Kamala price hikes have cost the average American family $28,000?”
— Donald Trump, in a TikTok video, Aug. 15
Get the latest election news and results
“As a result of Kamala’s inflation price hikes, they’ve cost the typical household a total of $28,000. These are numbers coming from the government. They are not coming from me.”
— Trump, media event in Bedminster, N.J., Aug. 15"""

response = few_shot_claim_chain.run(input=quoted_input)
print(response)

Here are the notable claims with their excerpts and restatements:

1. Excerpt: “Did you know the Kamala price hikes have cost the average American family $28,000?”
Restatement: The average American family has been affected by price hikes under Kamala Harris's leadership.

2. Excerpt: “As a result of Kamala’s inflation price hikes, they’ve cost the typical household a total of $28,000. These are numbers coming from the government. They are not coming from me.”
Restatement: The typical household has been affected by price hikes under Kamala Harris's leadership to the tune of $28,000, according to government data.

3. Excerpt: 'who literally wrote the book on annotation (“Annotation,” MIT Press)'
Restatement: Remi Kalir and Antero Garcia wrote a book on annotation.

Note that there are three notable claims in this text.


In [69]:
response = few_shot_claim_chain.run(input="Who's there?")
print(response)

No notable claims found.


In [70]:
response = few_shot_claim_chain.run(input="'I shot the sheriff, but I did not shoot the deputy'\n- Bob Marley")
print(response)

Excerpt: 'I shot the sheriff, but I did not shoot the deputy'
Restatement: Bob Marley said he killed the sheriff, but not the deputy.
